In [40]:
import numpy as np
from scipy.optimize import minimize

In [14]:
def sim_factor_model(loadings, specific_variance, mu, nsim=1, verbose=True):
    """
    Parameters
    ---
        loadings:           (p, k) matrix
        specific_variance:  (p, p) diagonal matrix, with specific variances on diagonals
        mu:                 (p, 1) vector of means
        nsim:               How many observations should be simulated

    Returns
    ---
        (n, p) matrix of observations from the specified factor model

    """
    k = loadings.shape[1]
    p = specific_variance.shape[0]
    if verbose:
        print(f"{k=} {p=}")

    X = []
    for _ in range(nsim):
        factor_vector = np.random.multivariate_normal(np.zeros(k), np.eye(k))
        u = np.random.multivariate_normal(np.zeros(p), np.diag(specific_variance))

        X.append(loadings @ factor_vector + u + mu)

    return np.array(X)

Goal is to minimize:
$$
tr((\hat{\Lambda}\hat{\Lambda}' + \Psi)^{-1} S) - log(|(\hat{\Lambda}\hat{\Lambda}' + \Psi)^{-1} S|) \quad \text{w.r.t. $\Psi$}
$$


Step by step:
1. Calculate $S^* = \Psi^{-1/2} S \Psi^{-1/2}$, $S^* = \Gamma \Theta \Gamma'$
2. Get eigenvectors $\gamma_{(i)}$ and values, $\theta_i$
3. Now $\Lambda^*$ has columns $c_i \lambda_{(i)}, \quad c_i = \sqrt{\max(\theta_i - 1, 0)}$
4. $\hat\Lambda = \Psi^{1/2} \Lambda^*$
5. Calculate the object function $\text{tr}((\hat\Lambda \hat\Lambda' + \Psi)^{-1} S) - log(|(\hat\Lambda \hat\Lambda' + \Psi)^{-1}S|)$

In [82]:
# Generate synthetic data
loadings = np.array([[2, 2, 2, 2, 2]]).T
specific_variance = np.array([2, 2, 10, 1, 1])
mu = np.array([10, 20, 30, 40, 50])

X = sim_factor_model(loadings, specific_variance, mu, nsim=10**4)
X

k=1 p=5


array([[ 8.94179311, 15.73306396, 29.84334347, 39.29370075, 48.24471971],
       [10.06426598, 21.7329725 , 32.79159805, 39.97726036, 51.49408432],
       [10.73336775, 19.54426817, 34.01919317, 41.64142892, 52.02406499],
       ...,
       [11.18569532, 21.62753465, 30.84192875, 42.309292  , 51.37909262],
       [ 8.55844209, 19.37005451, 29.09523596, 38.24747044, 49.38466488],
       [ 9.7489491 , 19.34510781, 23.36799554, 41.67228371, 51.55115352]])

1. Calculate $S^*$

In [83]:
S = np.cov(X.T)
Psi = np.diag(specific_variance)
Psi_sq_inv = np.linalg.inv(Psi ** 0.5)
S_star = Psi_sq_inv @ S @ Psi_sq_inv

2. Get eigenvectors and eigenvalues

In [84]:
eigval, eigvec = np.linalg.eig(S_star)

3. Construct $\Lambda^*$.

Here we must choose k. Lets choose k = 1, like the underlying model that generated the data. (the `loadings` used to simulate is (5, 1) = (p, k))

In [85]:
lambda_star = max(eigval[0] - 1, 0) ** 0.5 * eigvec[:,0]
lambda_star

array([-1.41420033, -1.41170534, -0.63232112, -1.98004082, -1.99641079])

4. Contstruct $\hat\Lambda = \Psi^{1/2} \Lambda^*$

In [86]:
lambda_hat = Psi ** 0.5 @ lambda_star
lambda_hat

array([-1.99998128, -1.99645284, -1.99957496, -1.98004082, -1.99641079])

5. Calculate $\text{tr}((\hat\Lambda \hat\Lambda' + \Psi)^{-1} S) - log(|(\hat\Lambda \hat\Lambda' + \Psi)^{-1}S|)$

In [87]:
internal = np.linalg.inv(lambda_hat @ lambda_hat + Psi) @ S
result = np.trace(internal) - np.log(np.linalg.det(internal))
result

5.7629985890706275

Having done this, lets construct a function that takes an arbitrary (D, 1) array of specific variances and calculates the objective function

In [96]:
def calculate_objective(specific_variance, X_data):
    # Step 1
    S = np.cov(X_data.T)
    Psi = np.diag(specific_variance)
    Psi_sq_inv = np.linalg.inv(Psi ** 0.5)
    S_star = Psi_sq_inv @ S @ Psi_sq_inv

    # Step 2
    eigval, eigvec = np.linalg.eig(S_star)

    # Step 3
    lambda_star = max(eigval[0] - 1, 0) ** 0.5 * eigvec[:,0]

    # Step 4
    lambda_hat = Psi ** 0.5 @ lambda_star

    # Step 5
    internal = np.linalg.inv(lambda_hat @ lambda_hat.T + Psi) @ S
    result = np.trace(internal) - np.log(np.linalg.det(internal))

    return result

In [89]:
specific_variance = np.array([2, 2, 10, 1, 1])
calculate_objective(specific_variance, X)

5.7629985890706275

This matches our manual step by step!

Now lets minimize the objective function. Note that the minimization algorithm works best with variables in $\R$, however $\psi_{ii} \geq 0$. We circumvent this by optimizing with $\alpha_i \in \R \rightarrow \psi_{ii} = \exp(\alpha_i) \in [0, \infty [$

In [ ]:
x_0_guess = np.array([2, 2, 10, 1, 1])
problem = minimize(fun=lambda x: calculate_objective(np.exp(x), X_data=X),
                   x0=x_0_guess)

  message: Optimization terminated successfully.
  success: True
   status: 0
      fun: 5.762284299942876
        x: [ 6.930e-01  7.248e-01  2.329e+00 -1.370e-02  1.093e-02]
      nit: 13
      jac: [ 2.146e-06 -3.517e-06 -1.132e-06 -1.073e-06 -6.557e-07]
 hess_inv: [[ 1.235e+00  1.387e-01 ... -1.273e-01 -1.046e-01]
            [ 1.387e-01  1.128e+00 ... -2.771e-01 -1.134e-01]
            ...
            [-1.273e-01 -2.771e-01 ...  2.382e+00 -6.615e-01]
            [-1.046e-01 -1.134e-01 ... -6.615e-01  2.419e+00]]
     nfev: 102
     njev: 17

In [118]:
psi_hat = np.exp(problem.x)
psi_hat

array([1.96246262, 2.02910606, 1.94914811, 0.9880357 , 1.01230155])

A simple change in step 3 lets us add the option to specify k number of factors.

In [116]:
def calculate_objective(specific_variance, X_data, k):
    # Step 1
    S = np.cov(X_data.T)
    Psi = np.diag(specific_variance)
    Psi_sq_inv = np.linalg.inv(Psi ** 0.5)
    S_star = Psi_sq_inv @ S @ Psi_sq_inv

    # Step 2
    eigval, eigvec = np.linalg.eig(S_star)

    # Step 3
    lambda_star = []
    for i in range(k):
        lambda_star.append(max(eigval[i] - 1, 0) ** 0.5 * eigvec[:,i])
    lambda_star = np.array(lambda_star).T

    # Step 4
    lambda_hat = Psi ** 0.5 @ lambda_star

    # Step 5
    internal = np.linalg.inv(lambda_hat @ lambda_hat.T + Psi) @ S
    result = np.trace(internal) - np.log(np.linalg.det(internal))

    return result

Let's generate some data that actually has k = 2 factors.

In [123]:
loadings = np.array([[2, 2, 2, 2, 2],
                     [1, 1, 0, -1, -1]]).T
specific_variance = np.array([2, 2, 10, 1, 1])
mu = np.array([10, 20, 30, 40, 50])

X = sim_factor_model(loadings, specific_variance, mu, nsim=10**5)

k=2 p=5


In [124]:
x_0_guess = np.array([2, 2, 10, 1, 1])
problem = minimize(fun=lambda x: calculate_objective(np.exp(x), X_data=X, k=2),
         x0=x_0_guess)
problem

  message: Optimization terminated successfully.
  success: True
   status: 0
      fun: 5.000000019451339
        x: [ 7.030e-01  6.721e-01  2.301e+00 -9.198e-03  1.159e-02]
      nit: 25
      jac: [-1.192e-07 -5.960e-08  7.749e-07  3.576e-06  5.007e-06]
 hess_inv: [[ 1.720e+01 -1.539e+01 ...  2.299e-01 -2.036e-02]
            [-1.539e+01  1.737e+01 ... -4.822e-01  2.490e-01]
            ...
            [ 2.299e-01 -4.822e-01 ...  3.946e+01 -3.476e+01]
            [-2.036e-02  2.490e-01 ... -3.476e+01  3.356e+01]]
     nfev: 174
     njev: 29

Then estimates are:

In [138]:
psi_hat = np.diag(np.exp(problem.x))

k = 2
S = np.cov(X.T)
Psi_sq_inv = np.linalg.inv(psi_hat ** 0.5)
S_star = Psi_sq_inv @ S @ Psi_sq_inv
eigval, eigvec = np.linalg.eig(S_star)
lambda_star = []
for i in range(2):
    lambda_star.append(max(eigval[i] - 1, 0) ** 0.5 * eigvec[:,i])
lambda_star = np.array(lambda_star).T
lambda_hat = psi_hat ** 0.5 @ lambda_star

print("Estimated psi: ", psi_hat, 
      "",
      "Estimated lambda:", lambda_hat,
      sep="\n")

Estimated psi: 
[[2.01971318 0.         0.         0.         0.        ]
 [0.         1.95827486 0.         0.         0.        ]
 [0.         0.         9.97996892 0.         0.        ]
 [0.         0.         0.         0.99084424 0.        ]
 [0.         0.         0.         0.         1.01165497]]

Estimated lambda:
[[ 1.75415117  1.36202321]
 [ 1.76979477  1.40110577]
 [ 1.95270123  0.40536224]
 [ 2.15384765 -0.59007522]
 [ 2.15466319 -0.58493958]]
